In [36]:
from langchain_ollama import ChatOllama 
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

In [37]:
model = ChatOllama(model = 'llama3.2:3b')

In [38]:
class EvaluationSchema(BaseModel):

    feedback: str = Field(description='Detailed Feedback for the essay')
    score: int = Field(description='Score out of 10', ge=0, le=10)

In [39]:
structured_model = model.with_structured_output(EvaluationSchema)

In [40]:
essay = """India is rapidly emerging as one of the most influential players in the global Artificial Intelligence (AI) ecosystem. With its vast talent pool, strong IT industry, expanding startup culture, and supportive government initiatives, India is positioning itself not only as a consumer of AI technologies but also as a global innovator and leader.
One of India’s greatest strengths in AI lies in its human capital. The country produces millions of STEM graduates every year, and Indian engineers and researchers contribute significantly to global AI advancements. Indian professionals play key roles in leading technology companies worldwide, driving innovation in machine learning, natural language processing, computer vision, and generative AI.
The Government of India has also taken proactive steps to promote AI development. Initiatives such as the National AI Strategy and Digital India aim to leverage AI for inclusive growth. AI is being actively applied in sectors like healthcare (disease detection and telemedicine), agriculture (crop prediction and smart farming), education (personalized learning), governance (smart cities and digital services), and finance (fraud detection and digital payments). These applications demonstrate India’s focus on using AI for social impact and large-scale public benefit.
India’s startup ecosystem is another major driver of AI growth. Thousands of AI-based startups are developing solutions in fintech, healthtech, edtech, climate tech, and enterprise automation. The combination of affordable data, large-scale digital infrastructure like Aadhaar and UPI, and a growing digital population gives India a unique advantage in building scalable AI systems.
However, challenges remain. Issues such as data privacy, ethical AI governance, infrastructure gaps, and the digital divide must be addressed to ensure responsible and inclusive AI development. Strengthening research funding, encouraging industry–academia collaboration, and building advanced computing infrastructure will be crucial for India to compete with global AI leaders.
In conclusion, India plays a significant and evolving role in the global AI landscape. By leveraging its talent, digital infrastructure, and innovation ecosystem, India has the potential to become a global AI powerhouse while ensuring that technology benefits all sections of society."""

In [50]:
essay2 = """
India role in AI is very big and also small sometimes. India is country with many peoples and many computer students. AI is Artificial Inteligence and it is machine doing thinking like human brain but not fully brain. India is trying to become powerful in AI but still learning many things.

In India many students study engineering and coding and they make AI projects. Government also doing programs for AI development and digital things. Some companies in India are making AI apps for health, farming, and banking. Farmers using AI for crop and weather but still many village not have good internet.

India also have problems like less money for research and not enough big computers for AI training. Some people also scared that AI will take jobs. But India want to grow and compete with big countries like USA and China in AI field.

So India role in AI is growing but not perfect. If India work more hard and fix problems, then India can become strong in AI in future.
"""

In [41]:
prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10. \n{essay}'

structured_model.invoke(prompt).score

8

In [51]:
class UPSCState(TypedDict):

    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [52]:
def evaluate_language(state: UPSCState):
    
    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10. \n {state['essay']}'

    output = structured_model.invoke(prompt)
    
    return {'language_feedback': output.feedback, 'individual_scores':[output.score]}

In [53]:
def evaluate_analysis(state: UPSCState):
    
    prompt = f'Evaluate the depth of analysis the following essay and provide a feedback and assign a score out of 10. \n {state['essay']}'

    output = structured_model.invoke(prompt)
    
    return {'analysis_feedback': output.feedback, 'individual_scores':[output.score]}

In [54]:
def evaluate_thought(state: UPSCState):
    
    
    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10. \n {state['essay']}'

    output = structured_model.invoke(prompt)
    
    return {'clarity_feedback': output.feedback, 'individual_scores':[output.score]}

In [55]:
def final_evaluation(state: UPSCState):

    # summary feedback
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n Clarity of thought feedback - {state["clarity_feedback"]}' 
    overall_feedback = model.invoke(prompt).content

    # Average calculate
    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback': overall_feedback, "avg_score": avg_score}

In [56]:
graph = StateGraph(UPSCState)

graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought', evaluate_thought)
graph.add_node('final_evaluation', final_evaluation)

# Create edges 
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')
graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')
graph.add_edge('final_evaluation', END)

#  Compile
workflow = graph.compile()


In [58]:
# workflow

In [59]:
initial_state = {
    'essay' : essay
}

workflow.invoke(initial_state)

{'essay': 'India is rapidly emerging as one of the most influential players in the global Artificial Intelligence (AI) ecosystem. With its vast talent pool, strong IT industry, expanding startup culture, and supportive government initiatives, India is positioning itself not only as a consumer of AI technologies but also as a global innovator and leader.\nOne of India’s greatest strengths in AI lies in its human capital. The country produces millions of STEM graduates every year, and Indian engineers and researchers contribute significantly to global AI advancements. Indian professionals play key roles in leading technology companies worldwide, driving innovation in machine learning, natural language processing, computer vision, and generative AI.\nThe Government of India has also taken proactive steps to promote AI development. Initiatives such as the National AI Strategy and Digital India aim to leverage AI for inclusive growth. AI is being actively applied in sectors like healthcare 

In [60]:
initial_state = {
    'essay' : essay2
}

workflow.invoke(initial_state)

{'essay': '\nIndia role in AI is very big and also small sometimes. India is country with many peoples and many computer students. AI is Artificial Inteligence and it is machine doing thinking like human brain but not fully brain. India is trying to become powerful in AI but still learning many things.\n\nIn India many students study engineering and coding and they make AI projects. Government also doing programs for AI development and digital things. Some companies in India are making AI apps for health, farming, and banking. Farmers using AI for crop and weather but still many village not have good internet.\n\nIndia also have problems like less money for research and not enough big computers for AI training. Some people also scared that AI will take jobs. But India want to grow and compete with big countries like USA and China in AI field.\n\nSo India role in AI is growing but not perfect. If India work more hard and fix problems, then India can become strong in AI in future.\n',
 '